In [2]:
from gurobipy import *
import pandas as pd
import numpy as np
import math
from itertools import combinations,product

In this notebook, we will use the DFJ formulation with subtour cuts to output the optimal tour.  

In [3]:
# Function will convert our datfile of x,y coordinates into dataframe: vertices
def TSP_vertices(file_name):
    data  = open(file_name, 'r')
    lines = data.readlines()[1:]
    x = []
    y = []
    for line in lines:
        p = line.split()
        x.append(float(p[0]))
        y.append(float(p[1]))
    data.close()
    # Transform into dataframe
    vertices = pd.DataFrame({'x':x, 'y':y})
    
    return vertices

In [4]:
# Will convert Euclidean distance for our coordinates as cost components for model
def euclideanDistMatrix(TSP):
    euclidean_cost = []
    for i in range(len(TSP)):
        row_distances = []
        for j in range(len(TSP)):
            row_distances.append(math.sqrt( (TSP['x'][i]-TSP['x'][j])**2 + (TSP['y'][i]-TSP['y'][j])**2 ))
        euclidean_cost.append(row_distances)
    return euclidean_cost  

In [1]:
### This function will be called from the main DFJ function. It will create the subour constraints as need be.
def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        # make a list of edges selected in the solution
        vals = model.cbGetSolution(model._vars)
        selected = tuplelist((i, j) for i, j in model._vars.keys()
                                if vals[i, j] > 0.5)
        # find the shortest cycle in the selected edge list
        tour = subtour(selected)
        if len(tour) < len(selected)//2:
            # add subtour elimination constr. for every pair of cities in tour
            model.cbLazy(quicksum(model._vars[i, j]
                                     for i, j in combinations(tour, 2))
                         <= len(tour)-1)

In [2]:
### This function will be called by subtourlim() to find the shortest subtour.
def subtour(edges):
    unvisited = list(range(len(edges)//2))
    cycle = range(len(edges)//2 + 1)  # initial length has 1 more city
    while unvisited:  # true if list is non-empty
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j in unvisited]
        if len(cycle) > len(thiscycle):
            cycle = thiscycle
    return cycle

In [3]:
### Function for the Dantzig Fulkerson and Johnson Formulation

def DFJ_Formulation(file_name):
    # Set up the model
    model = Model('DFJ')
    
    # Read in file name through function TSP_vertices and use euclideanDistMatrix to calculate all i to j traverse costs
    DFJvertices = TSP_vertices(file_name)
    cost = euclideanDistMatrix(DFJvertices)

    # Put all the vertices in a list: V
    V = []
    for i in range(len(DFJvertices)):
        V.append(i)

    # the dist dictionary has the arcs and their costs
    dist = {(v1, v2): cost[v1][v2] for v1, v2 in product(V, V) if v1 != v2}

    # Set up the decision variable and objective function
    vars = model.addVars(dist.keys(), obj=dist, vtype=GRB.BINARY, name='e')
    model.modelSense = GRB.MINIMIZE
    model.setObjective(quicksum(vars[i,j]*cost[i][j] for i in V for j in V if i != j))

    # Constraint 1: Ensuring the costs of traversing is symmetric
    for i, j in vars.keys():
        vars[j, i] = vars[i, j]

    # Constraint 2: Degree of two for each vertex
    model.addConstrs(vars.sum(i, '*') == 2 for i in range(len(V)))

    # Solve for optimal solution with the function subtourelim called if solution contains subtours. 
    model._vars = vars
    model.Params.lazyConstraints = 1
    model.Params.TimeLimit = 600
    model.optimize(subtourelim)
    
    print('The runtime was', model.Runtime, 'seconds.')
    
    ### UNCOMMENT LINE BELOW if you want to save a file of the formulation. Open with Microsoft Word.
    model.write('DFJ.lp')

In [7]:
### Function for the Linear Relaxation of the Dantzig Fulkerson and Johnson Formulation

def DFJ_Formulation_LR(file_name):
    # Set up the model
    model = Model('DFJ_LR')
    
    # Read in file name through function TSP_vertices and use euclideanDistMatrix to calculate all i to j traverse costs
    DFJvertices = TSP_vertices(file_name)
    cost = euclideanDistMatrix(DFJvertices)

    # Put all the vertices in a list: V
    V = []
    for i in range(len(DFJvertices)):
        V.append(i)

    # the dist dictionary has the arcs and their costs
    dist = {(v1, v2): cost[v1][v2] for v1, v2 in product(V, V) if v1 != v2}

    # Set up the decision variable and objective function
    vars = model.addVars(dist.keys(), obj=dist, lb=0, ub=1, vtype=GRB.CONTINUOUS, name='e')
    model.modelSense = GRB.MINIMIZE
    model.setObjective(quicksum(vars[i,j]*cost[i][j] for i in V for j in V if i != j))

    # Constraint 1: Ensuring the costs of traversing is symmetric
    for i, j in vars.keys():
        vars[j, i] = vars[i, j]

    # Constraint 2: Degree of two for each vertex
    model.addConstrs(vars.sum(i, '*') == 2 for i in range(len(V)))

    # Solve for optimal solution with the function subtourelim called if solution contains subtours. 
    model._vars = vars
    model.Params.lazyConstraints = 1
    model.Params.TimeLimit = 600
    model.optimize(subtourelim)
    
    print('The runtime was', model.Runtime, 'seconds.')
    
    ### UNCOMMENT LINE BELOW if you want to save a file of the formulation. Open with Microsoft Word.
    # model.write('DFJ_LR.lp')

The following are the DFJ formulation integer and Linear Relaxation solutions for two instances of size 15.

In [8]:
DFJ_Formulation('TSP_instance_n_15_s_1.dat')

Using license file /Users/student/gurobi.lic
Academic license - for non-commercial use only
Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 15 rows, 210 columns and 210 nonzeros
Model fingerprint: 0x008cce47
Variable types: 0 continuous, 210 integer (210 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.00s
Presolved: 15 rows, 210 columns, 210 nonzeros
Variable types: 0 continuous, 210 integer (210 binary)

Root relaxation: objective 6.155745e+01, 25 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0            

In [9]:
DFJ_Formulation_LR('TSP_instance_n_15_s_1.dat')

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 15 rows, 210 columns and 210 nonzeros
Model fingerprint: 0x84e23b97
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.01s
Presolved: 15 rows, 210 columns, 210 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.8305327e+00   2.700000e+01   0.000000e+00      0s
      20    6.0360068e+01   0.000000e+00   0.000000e+00      0s

Solved in 20 iterations and 0.04 seconds
Optimal objective  6.036006804e+01

User-callback calls 45, time in user-callback 0.00 sec
The runtime was 0.046627044677734375 seconds.


In [10]:
DFJ_Formulation('TSP_instance_n_15_s_2.dat')

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 15 rows, 210 columns and 210 nonzeros
Model fingerprint: 0x61fbb405
Variable types: 0 continuous, 210 integer (210 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.00s
Presolved: 15 rows, 210 columns, 210 nonzeros
Variable types: 0 continuous, 210 integer (210 binary)

Root relaxation: objective 6.322129e+01, 15 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   67.51179    0    6          -   67.51179      -     -    0s
*    0     0               0      71.8241

In [11]:
DFJ_Formulation_LR('TSP_instance_n_15_s_2.dat')

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 15 rows, 210 columns and 210 nonzeros
Model fingerprint: 0x306d13a4
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.01s
Presolved: 15 rows, 210 columns, 210 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.0155205e+00   2.700000e+01   0.000000e+00      0s
      15    6.3221287e+01   0.000000e+00   0.000000e+00      0s

Solved in 15 iterations and 0.08 seconds
Optimal objective  6.322128723e+01

User-callback calls 40, time in user-callback 0.00 sec
The runtime was 0.09090900421142578 seconds.


The following are the DFJ formulation integer and Linear Relaxation solutions for two instances of size 20.

In [12]:
DFJ_Formulation('TSP_instance_n_20_s_1.dat')

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 20 rows, 380 columns and 380 nonzeros
Model fingerprint: 0x01d3b778
Variable types: 0 continuous, 380 integer (380 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.00s
Presolved: 20 rows, 380 columns, 380 nonzeros
Variable types: 0 continuous, 380 integer (380 binary)

Root relaxation: objective 6.268073e+01, 29 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   62.68073    0    6          -   62.68073      -     -    0s
     0     0   67.26897    0    6        

In [13]:
DFJ_Formulation_LR('TSP_instance_n_20_s_1.dat')

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 20 rows, 380 columns and 380 nonzeros
Model fingerprint: 0xd683f8f2
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.02s
Presolved: 20 rows, 380 columns, 380 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.4768870e-01   3.700000e+01   0.000000e+00      0s
      29    5.9083047e+01   0.000000e+00   0.000000e+00      0s

Solved in 29 iterations and 0.03 seconds
Optimal objective  5.908304727e+01

User-callback calls 54, time in user-callback 0.00 sec
The runtime was 0.03957200050354004 seconds.


In [14]:
DFJ_Formulation('TSP_instance_n_20_s_2.dat')

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 20 rows, 380 columns and 380 nonzeros
Model fingerprint: 0xd7561c58
Variable types: 0 continuous, 380 integer (380 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.00s
Presolved: 20 rows, 380 columns, 380 nonzeros
Variable types: 0 continuous, 380 integer (380 binary)

Root relaxation: objective 6.636115e+01, 27 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   66.36115    0    6          -   66.36115      -     -    0s
     0     0   74.20562    0    8        

In [15]:
DFJ_Formulation_LR('TSP_instance_n_20_s_2.dat')

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 20 rows, 380 columns and 380 nonzeros
Model fingerprint: 0x388d7e07
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.01s
Presolved: 20 rows, 380 columns, 380 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.8162658e+00   3.700000e+01   0.000000e+00      0s
      27    6.6361148e+01   0.000000e+00   0.000000e+00      0s

Solved in 27 iterations and 0.03 seconds
Optimal objective  6.636114771e+01

User-callback calls 52, time in user-callback 0.00 sec
The runtime was 0.041040897369384766 seconds.


The following are the DFJ formulation integer and Linear Relaxation solutions for two instances of size 25.

In [16]:
DFJ_Formulation('TSP_instance_n_25_s_1.dat')

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 25 rows, 600 columns and 600 nonzeros
Model fingerprint: 0xade7cb19
Variable types: 0 continuous, 600 integer (600 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Found heuristic solution: objective 207.5491997
Presolve time: 0.00s
Presolved: 25 rows, 600 columns, 600 nonzeros
Variable types: 0 continuous, 600 integer (600 binary)

Root relaxation: objective 7.336774e+01, 46 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   73.36774    0    6  207.54920   73.36774  64.7%     -

In [17]:
DFJ_Formulation_LR('TSP_instance_n_25_s_1.dat')

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 25 rows, 600 columns and 600 nonzeros
Model fingerprint: 0x69190a1a
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.02s
Presolved: 25 rows, 600 columns, 600 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.4768870e-01   4.700000e+01   0.000000e+00      0s
      37    7.3367739e+01   0.000000e+00   0.000000e+00      0s

Solved in 37 iterations and 0.04 seconds
Optimal objective  7.336773856e+01

User-callback calls 61, time in user-callback 0.00 sec
The runtime was 0.0563509464263916 seconds.


In [18]:
DFJ_Formulation('TSP_instance_n_25_s_2.dat')

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 25 rows, 600 columns and 600 nonzeros
Model fingerprint: 0x188daa70
Variable types: 0 continuous, 600 integer (600 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Found heuristic solution: objective 282.7364170
Presolve time: 0.00s
Presolved: 25 rows, 600 columns, 600 nonzeros
Variable types: 0 continuous, 600 integer (600 binary)

Root relaxation: objective 7.531660e+01, 34 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   76.46829    0    6  282.73642   76.46829  73.0%     -

In [19]:
DFJ_Formulation_LR('TSP_instance_n_25_s_2.dat')

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 25 rows, 600 columns and 600 nonzeros
Model fingerprint: 0x87661d29
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.01s
Presolved: 25 rows, 600 columns, 600 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.8162658e+00   4.700000e+01   0.000000e+00      0s
      35    7.5316602e+01   0.000000e+00   0.000000e+00      0s

Solved in 35 iterations and 0.02 seconds
Optimal objective  7.531660163e+01

User-callback calls 60, time in user-callback 0.00 sec
The runtime was 0.03578495979309082 seconds.


The following are the DFJ formulation integer and Linear Relaxation solutions for two instances of size 30.

In [20]:
DFJ_Formulation('TSP_instance_n_30_s_1.dat')

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 30 rows, 870 columns and 870 nonzeros
Model fingerprint: 0x041ab82c
Variable types: 0 continuous, 870 integer (870 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.00s
Presolved: 30 rows, 870 columns, 870 nonzeros
Variable types: 0 continuous, 870 integer (870 binary)

Root relaxation: objective 8.482838e+01, 51 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   84.82838    0    6          -   84.82838      -     -    0s
     0     0   86.91465    0    6        

In [21]:
DFJ_Formulation_LR('TSP_instance_n_30_s_1.dat')

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 30 rows, 870 columns and 870 nonzeros
Model fingerprint: 0xeaeccd5e
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.02s
Presolved: 30 rows, 870 columns, 870 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.4768870e-01   5.700000e+01   0.000000e+00      0s
      52    8.4828381e+01   0.000000e+00   0.000000e+00      0s

Solved in 52 iterations and 0.03 seconds
Optimal objective  8.482838102e+01

User-callback calls 77, time in user-callback 0.00 sec
The runtime was 0.039537906646728516 seconds.


In [22]:
DFJ_Formulation('TSP_instance_n_30_s_2.dat')

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 30 rows, 870 columns and 870 nonzeros
Model fingerprint: 0x59dfac2f
Variable types: 0 continuous, 870 integer (870 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.00s
Presolved: 30 rows, 870 columns, 870 nonzeros
Variable types: 0 continuous, 870 integer (870 binary)

Root relaxation: objective 8.179642e+01, 50 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   81.79642    0    6          -   81.79642      -     -    0s
     0     0   86.43678    0    8        

In [23]:
DFJ_Formulation_LR('TSP_instance_n_30_s_2.dat')

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 30 rows, 870 columns and 870 nonzeros
Model fingerprint: 0xd8ddf4ac
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.01s
Presolved: 30 rows, 870 columns, 870 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.8162658e+00   5.700000e+01   0.000000e+00      0s
      41    8.1216436e+01   0.000000e+00   0.000000e+00      0s

Solved in 41 iterations and 0.03 seconds
Optimal objective  8.121643613e+01

User-callback calls 66, time in user-callback 0.00 sec
The runtime was 0.03857898712158203 seconds.


The following are the DFJ formulation integer and Linear Relaxation solutions for two instances of size 50.

In [24]:
DFJ_Formulation('TSP_instance_n_50_s_1.dat')

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 50 rows, 2450 columns and 2450 nonzeros
Model fingerprint: 0x2dc40f18
Variable types: 0 continuous, 2450 integer (2450 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Found heuristic solution: objective 506.4764282
Presolve time: 0.01s
Presolved: 50 rows, 2450 columns, 2450 nonzeros
Variable types: 0 continuous, 2450 integer (2450 binary)

Root relaxation: objective 1.067319e+02, 76 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  106.73195    0   20  506.47643  106.73195  78.

In [25]:
DFJ_Formulation_LR('TSP_instance_n_50_s_1.dat')

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 50 rows, 2450 columns and 2450 nonzeros
Model fingerprint: 0x9c3c726e
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.01s
Presolved: 50 rows, 2450 columns, 2450 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.4768870e-01   9.700000e+01   0.000000e+00      0s
      83    1.0673195e+02   0.000000e+00   0.000000e+00      0s

Solved in 83 iterations and 0.03 seconds
Optimal objective  1.067319456e+02

User-callback calls 108, time in user-callback 0.00 sec
The runtime was 0.039759159088134766 seconds.


In [26]:
DFJ_Formulation('TSP_instance_n_50_s_2.dat')

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 50 rows, 2450 columns and 2450 nonzeros
Model fingerprint: 0xef43f43d
Variable types: 0 continuous, 2450 integer (2450 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Found heuristic solution: objective 559.1768340
Presolve time: 0.00s
Presolved: 50 rows, 2450 columns, 2450 nonzeros
Variable types: 0 continuous, 2450 integer (2450 binary)

Root relaxation: objective 9.331014e+01, 61 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   98.62893    0    8  559.17683   98.62893  82.

In [27]:
DFJ_Formulation_LR('TSP_instance_n_50_s_2.dat')

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 50 rows, 2450 columns and 2450 nonzeros
Model fingerprint: 0xce795a9c
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.01s
Presolved: 50 rows, 2450 columns, 2450 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.7459361e-01   9.700000e+01   0.000000e+00      0s
      60    9.3310137e+01   0.000000e+00   0.000000e+00      0s

Solved in 60 iterations and 0.03 seconds
Optimal objective  9.331013666e+01

User-callback calls 85, time in user-callback 0.00 sec
The runtime was 0.037866830825805664 seconds.


The following are the DFJ formulation integer and Linear Relaxation solutions for two instances of size 70.

In [28]:
DFJ_Formulation('TSP_instance_n_70_s_1.dat')

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 70 rows, 4830 columns and 4830 nonzeros
Model fingerprint: 0x914775bc
Variable types: 0 continuous, 4830 integer (4830 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.01s
Presolved: 70 rows, 4830 columns, 4830 nonzeros
Variable types: 0 continuous, 4830 integer (4830 binary)

Root relaxation: objective 1.260454e+02, 114 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  126.04543    0   20          -  126.04543      -     -    0s
     0     0  128.69202    0   1

In [29]:
DFJ_Formulation_LR('TSP_instance_n_70_s_1.dat')

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 70 rows, 4830 columns and 4830 nonzeros
Model fingerprint: 0x7c9d5658
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.02s
Presolved: 70 rows, 4830 columns, 4830 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.4768870e-01   1.370000e+02   0.000000e+00      0s
     105    1.2604543e+02   0.000000e+00   0.000000e+00      0s

Solved in 105 iterations and 0.03 seconds
Optimal objective  1.260454294e+02

User-callback calls 130, time in user-callback 0.00 sec
The runtime was 0.04131913185119629 seconds.


In [30]:
DFJ_Formulation('TSP_instance_n_70_s_2.dat')

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 70 rows, 4830 columns and 4830 nonzeros
Model fingerprint: 0x1263601d
Variable types: 0 continuous, 4830 integer (4830 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.01s
Presolved: 70 rows, 4830 columns, 4830 nonzeros
Variable types: 0 continuous, 4830 integer (4830 binary)

Root relaxation: objective 1.270792e+02, 99 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  127.07923    0   12          -  127.07923      -     -    0s
     0     0  129.04756    0    8

In [31]:
DFJ_Formulation_LR('TSP_instance_n_70_s_2.dat')

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 70 rows, 4830 columns and 4830 nonzeros
Model fingerprint: 0xc15d3c43
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.02s
Presolved: 70 rows, 4830 columns, 4830 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.7459361e-01   1.370000e+02   0.000000e+00      0s
     101    1.2707923e+02   0.000000e+00   0.000000e+00      0s

Solved in 101 iterations and 0.05 seconds
Optimal objective  1.270792343e+02

User-callback calls 126, time in user-callback 0.00 sec
The runtime was 0.05652308464050293 seconds.
